In [73]:
import pandas as pd

df = pd.read_csv("data/steam_data_with_time.csv", names=["Game", "GameID", "PlayerCount", "time"], header=None)
df[df['PlayerCount'] > 1000]

,Game,GameID,PlayerCount,time
3194,MechWarrior 5: Mercenaries,784080,2032.0,17:09:46
3305,DOOM Eternal,782330,1207.0,17:10:04
4273,War Robots,767560,1559.0,17:12:32
4328,Russian Fishing 4,766570,4176.0,17:12:41
4802,Vampire: The Masquerade - Bloodhunt,760160,1089.0,17:13:51
5970,Neverwinter Nights: Enhanced Edition,704450,1051.0,17:16:48
6086,March of Empires,702320,1096.0,17:17:06
8055,Cosmoteer: Starship Architect & Commander,799600,1205.0,17:21:43
9272,Overcooked! 2,728880,1447.0,17:25:34
12565,Crab Game,1782210,8650.0,17:37:54


In [78]:
df = pd.read_csv("data/games_to_ignore.csv", names=["Game"], header=None)


In [79]:
df

,Game


In [67]:
df.shape

(10638, 4)

In [50]:
import requests

def fetch_reviews(app_id):
    url = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Example usage
# reviews = fetch_reviews(2114740)  # Replace 2114740 with the actual GameID
fetch_reviews(782330)

{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 94770,
  'total_negative': 9125,
  'total_reviews': 103895},
 'reviews': [{'recommendationid': '146373402',
   'author': {'steamid': '76561198158383927',
    'num_games_owned': 0,
    'num_reviews': 22,
    'playtime_forever': 1327,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 1327,
    'last_played': 1625018340},
   'language': 'english',
   'review': 'We may not see a flawless FPS combat system like this for years',
   'timestamp_created': 1694833913,
   'timestamp_updated': 1694833913,
   'voted_up': True,
   'votes_up': 89,
   'votes_funny': 4,
   'weighted_vote_score': '0.819789707660675049',
   'comment_count': 9,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '146503855',
   'author': {'steamid